<a href="https://colab.research.google.com/github/Shubhammt/cs6910_assignment3/blob/main/seq2seq_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1: Model Building

In [ ]:
import numpy as np
import tensorflow 
import keras
import io
from math import log,log1p
from numpy import array,argmax
from keras.models import Model,load_model
from keras.layers import Input, LSTM, Dense, Embedding, GRU, Dropout, SimpleRNN
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from tensorflow.keras.optimizers import Adam

In [ ]:
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback   

In [ ]:
wandb.login()

True

In [ ]:
wandb.init(project="Assignment 3", entity="shubham-argha")

wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


In [ ]:
class seq2seq(object):
    def __init__(self,Type = 'rnn',input_embedding = 32, hid_layer_size = 32, lr= 1e-3, 
                   drop_prob=0.4,prediction ='greedy',number_of_epochs = 10, batch_size = 32,beam_width = 5,
                   number_of_encoders = 1, number_of_decoders= 1):

        self.Type = Type
        self.input_embedding = input_embedding
        self.hid_layer_size = hid_layer_size
        self.lr = lr
        self.drop_prob = drop_prob
        self.prediction = prediction
        self.number_of_epochs = number_of_epochs
        self.batch_size = batch_size
        self.beam_width = beam_width
        self.number_of_encoders = number_of_encoders
        self.number_of_decoders = number_of_decoders

    def fit(self,enc_inp, dec_inp, dec_tar,x_test, y_test):
        # Input Sequence
        enc_inps= Input(shape=(None, ),name = 'Enc_inputs')

        # Embedding Layer
        encoder_embedding=  Embedding(enc_tok_num, self.input_embedding , mask_zero = True,name = 'Enc_emb')(enc_inps)

        enc_outs= encoder_embedding

        if self.Type == 'lstm':

            enc_LSTM = LSTM(self.hid_layer_size, return_state=True,dropout = self.drop_prob, return_sequences=True, name="Enc_hidden_1")
            enc_outs, h, c = enc_LSTM(enc_outs)
            states= [h, c]

            # LSTM Layer 
            for i in range( 2, self.number_of_encoders +1):
                l= ('Enc_hidden_%d')%i
                enc_LSTM = LSTM(self.hid_layer_size, return_state=True,dropout = self.drop_prob, return_sequences=True, name=l)
                enc_outs, h, c = enc_LSTM(enc_outs,initial_state = states)
                states = [h, c]

        elif self.Type == 'gru':
            enc_GRU = GRU(self.hid_layer_size, return_state=True,dropout = self.drop_prob, return_sequences=True, name="Enc_hidden_1")
            enc_outs, h = enc_GRU(enc_outs)
            states = [h]

            # GRU Layer
            for i in range(2, self.number_of_encoders +1):
                l = ('Enc_hidden_%d') %i
                enc_GRU = GRU(self.hid_layer_size, return_state=True,dropout = self.drop_prob, return_sequences=True, name=l)
                enc_outs, h = enc_GRU(enc_outs, initial_state = states)
                states = [h]  

        elif self.Type == 'rnn':
            enc_rnn = SimpleRNN(self.hid_layer_size, return_state=True,dropout = self.drop_prob, return_sequences=True, name="Enc_hidden_1")
            enc_outs, h = enc_rnn(enc_outs)
            states = [h]
            
            # RNN Layer
            for i in range(2, self.number_of_encoders +1):
                l = ('Enc_hidden_%d') %i
                enc_rnn = SimpleRNN(self.hid_layer_size, return_state=True,dropout = self.drop_prob, return_sequences=True, name=l)
                enc_outs, h = enc_rnn(enc_outs, initial_state = states)
                states = [h]  

        # Decoder uses "encoder_states" as initial state
        dec_inps= Input(shape=(None,), name = 'Dec_inputs')
        decoder_embedding_layer = Embedding(dec_tok_num, self.hid_layer_size, mask_zero = True, name = 'Dec_emb')
        decoder_embedding = decoder_embedding_layer(dec_inps)
        dec_outs = decoder_embedding

        if self.Type == 'lstm':
            dec_LSTM = LSTM(self.hid_layer_size, return_sequences=True, return_state=True,dropout = self.drop_prob, name="Dec_hidden_1")
            dec_outs, _, _ = dec_LSTM(dec_outs, initial_state = states)

            for i in range(2, self.number_of_decoders +1):
                l = ('Dec_hidden_%d') %i

                dec_LSTM = LSTM(self.hid_layer_size, return_sequences=True, return_state=True,dropout = self.drop_prob, name=l)
                dec_outs, _, _ = dec_LSTM(dec_outs, initial_state = states)

        elif self.Type == 'gru':
            dec_GRU = GRU(self.hid_layer_size, return_sequences=True, return_state=True,dropout = self.drop_prob, name="Dec_hidden_1")
            dec_outs, _ = dec_GRU(dec_outs, initial_state = states)

            for i in range(2, self.number_of_decoders+1):
                l = ('Dec_hidden_%d') %i
                dec_GRU = GRU(self.hid_layer_size, return_sequences=True, return_state=True,dropout = self.drop_prob, name=l)
                dec_outs, _ = dec_GRU(dec_outs, initial_state = states)

        elif self.Type == 'rnn':
            dec_RNN = SimpleRNN(self.hid_layer_size, return_sequences=True, return_state=True,dropout = self.drop_prob, name="Dec_hidden_1")
            dec_outs, _ = dec_RNN(dec_outs, initial_state = states)

            for i in range(2, self.number_of_decoders+1):
                l = ('Dec_hidden_%d') %i
                dec_RNN = SimpleRNN(self.hid_layer_size, return_sequences=True, return_state=True,dropout = self.drop_prob, name=l)
                dec_outs, _ = dec_RNN(dec_outs, initial_state = states)

        den_dec= Dense(dec_tok_num, activation='softmax', name = 'dense')
        dec_outs = den_dec(dec_outs)


        # model takes encoder input and decoder input and outputs decoder output
        model = Model([enc_inps, dec_inps], dec_outs)

        model.summary()
        
        # Optimizer
        optimizer = Adam(lr=self.lr, beta_1=0.9, beta_2=0.999)
        model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])

        model.fit(
            [enc_inp, dec_inp],
            dec_tar,
            batch_size=self.batch_size,
            epochs=self.number_of_epochs,
            callbacks = [WandbCallback()]
            )


        enc_mod, dec_mod = self.inference(model)
        g_t = 0
        g_c = 0

        for i in range(len(val_src)):

            input_sequence = x_test[i : i + 1]
            rlt = self.dec_seq(enc_mod,dec_mod,input_sequence)
            tgt = y_test[i]
            tgt = tgt[1:len(tgt)-1]
            rlt = rlt[0:len(rlt)-1]
            
            if rlt.strip() == tgt.strip():
                g_c = g_c + 1

            g_t = g_t + 1
            accuracy_epoch = g_c/g_t

            if g_t % 50 == 0:
                wandb.log({'epoch_accuracy' : accuracy_epoch})
            
        val_accuracy = g_c/g_t
        wandb.log({'val_accuracy' : val_accuracy})


    def inference(self, model):

        enc_inps = model.input[0]  

        if self.Type == 'rnn' or self.Type == 'gru':
            enc_outs, encoder_h = model.get_layer('Enc_hidden_'+ str(self.number_of_encoders)).output
            states = [encoder_h]
            enc_mod = Model(enc_inps, states)

            dec_inps = model.input[1]  
            dec_outs = model.get_layer('Dec_emb')(dec_inps)
            inps_dec_states = []
            dec_states = []

            for i in range(1,self.number_of_decoders +1):
                decoder_state_input_h = keras.Input(shape=(self.hid_layer_size,))
                curr_states_inputs = [decoder_state_input_h]
                decoder = model.get_layer('Dec_hidden_'+ str(i))
                dec_outs, decoder_h = decoder(dec_outs, initial_state=curr_states_inputs)

                dec_states += [decoder_h]
                inps_dec_states += curr_states_inputs

        elif self.Type == 'lstm':
            enc_outs, encoder_h, encoder_c = model.get_layer('Enc_hidden_'+ str(self.number_of_encoders)).output 
            states = [encoder_h, encoder_c]
            enc_mod = Model(enc_inps, states)

            dec_inps = model.input[1] 
            dec_outs = model.get_layer('Dec_emb')(dec_inps)
            inps_dec_states = []
            dec_states = []

            for i in range(1,self.number_of_decoders +1):
                decoder_state_input_h = keras.Input(shape=(self.hid_layer_size,))
                decoder_state_input_c = keras.Input(shape=(self.hid_layer_size,))
                curr_states_inputs = [decoder_state_input_h, decoder_state_input_c]
                decoder = model.get_layer('Dec_hidden_'+ str(i))
                dec_outs, decoder_h, decoder_c = decoder(dec_outs, initial_state=curr_states_inputs)

                dec_states += [decoder_h, decoder_c]
                inps_dec_states += curr_states_inputs


        den_dec = model.get_layer('dense')
        dec_outs = den_dec(dec_outs)
        dec_mod = Model([dec_inps] + inps_dec_states, [dec_outs] + dec_states)

        return enc_mod,dec_mod
        

    def search_beam(self,data, k):
        seqs = [[list(), 0.0]]
        for r in data:
            a = list()
            for i in range(len(seqs)):
                seq, score = seqs[i]
                for j in range(len(r)):
                    c = [seq + [j], score - log(r[j])]
                    a.append(c)
        ord = sorted(a, key=lambda tup:tup[1])
        seqs = ord[:k]
        return seqs



    def dec_seq(self,enc_mod,dec_mod,input_sequence):
        #Input State Vectors
        val_states = [enc_mod.predict(input_sequence)] * self.number_of_decoders

        # Empty Target Sequence
        tar_sequence = np.zeros((1, 1))
        # Start Character
        tar_sequence[0, 0] = tok_ind_tar['B']

        # Sampling loop 
        flag = False
        decoded_sentence = ""

        while not flag:
            if self.Type == 'rnn' or self.Type == 'gru':
                d = dec_mod.predict([tar_sequence] + [val_states])
                output_tokens, val_states = d[0],d[1:]

            elif self.Type == 'lstm':  
                d = dec_mod.predict([tar_sequence] + val_states)
                output_tokens, val_states = d[0],d[1:]

            if self.prediction == 'greedy':
                width_beam = 1
            elif self.prediction == 'beam_search':
                width_beam = self.beam_width

            sampled_token_index = self.search_beam(output_tokens[0,:,:], width_beam)
            sampled_token_index = sampled_token_index[width_beam-1][0]

            # Token Sampling
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_char = rtci[sampled_token_index]
            decoded_sentence += sampled_char

            # Exit condition
            if sampled_char == 'E' or len(decoded_sentence) > max_decoder_seq_length:
                flag = True

            # tar_sequence update
            tar_sequence = np.zeros((1, 1))
            tar_sequence[0, 0] = sampled_token_index

        return decoded_sentence


# Question 2: Dataset and training

In [ ]:
train_dir = "../dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
dev_dir = "../dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
test_dir = "../dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"

In [ ]:
# Dataset
!yes | wget "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"

--2022-05-01 10:03:58--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar.1’

dakshina_dataset_v1 100%[===================>]   1.87G   128MB/s    in 25s     

2022-05-01 10:04:24 (75.2 MB/s) - ‘dakshina_dataset_v1.0.tar.1’ saved [2008340480/2008340480]



In [ ]:
# Unzip
!yes | tar xopf dakshina_dataset_v1.0.tar

In [ ]:
# Train, Test and Validation Data
!ls dakshina_dataset_v1.0/hi/lexicons

hi.translit.sampled.dev.tsv   hi.translit.sampled.train.tsv
hi.translit.sampled.test.tsv


In [ ]:
train_dir = "./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
dev_dir = "./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
test_dir = "./dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"

In [ ]:
# reads text document and returns a list of lists comprising the romanized and native versions of the words
def read(f):
    hindi = []
    latin = []
    with io.open(f, encoding ='utf-8') as f:
        for line in f:
            if '\t' not in line:
                continue
            tokens = line.rstrip().split("\t")
            latin.append(tokens[1])
            hindi.append(tokens[0])
    return latin, hindi

In [ ]:
tr_src, tr_tar = read(train_dir)
val_src, val_tar = read(dev_dir)
test_source, test_target = read(test_dir)

In [ ]:
print("Number of training samples: ", len(tr_src))
print("Number of validation samples: ", len(val_src))
print("Number of testing samples: ", len(test_source))

Number of training samples:  44204
Number of validation samples:  4358
Number of testing samples:  4502


In [ ]:
arr = np.arange(len(tr_src))
np.random.shuffle(arr)
arr1 = np.arange(len(val_src))
np.random.shuffle(arr1)

In [ ]:
char_inp = set()
char_tar = set()
ns_txt_inp = []
ns_txt_tar = []
ns_txt_inp_val = []
ns_txt_tar_val = []

In [ ]:
for (txt_inp, txt_tar) in zip(tr_src, tr_tar):
    # tab : "start sequence" character
    # \n  : "end sequence" character
    txt_tar = "B" + txt_tar + "E"
    ns_txt_inp.append(txt_inp)
    ns_txt_tar.append(txt_tar)

    for char in txt_inp:
        if char not in char_inp:
            char_inp.add(char)

    for char in txt_tar:
        if char not in char_tar:
            char_tar.add(char)


for (txt_inp, txt_tar) in zip(val_src, val_tar):
    # tab : "start sequence" character
    # \n  : "end sequence" character
    txt_tar = "B" + txt_tar + "E"
    ns_txt_inp_val.append(txt_inp)
    ns_txt_tar_val.append(txt_tar)
    for char in txt_inp:
        if char not in char_inp:
            char_inp.add(char)
    for char in txt_tar:
        if char not in char_tar:
            char_tar.add(char)

In [ ]:
inps_txt = []
tars_txt = []

for i in range(len(tr_src)):
    inps_txt.append(ns_txt_inp[arr[i]])
    tars_txt.append(ns_txt_tar[arr[i]])

val_input_texts = []
val_target_texts = []

for i in range(len(val_src)):
    val_input_texts.append(ns_txt_inp_val[arr1[i]])
    val_target_texts.append(ns_txt_tar_val[arr1[i]])

In [ ]:
char_inp.add(" ")
char_tar.add(" ")

char_inp = sorted(list(char_inp))
char_tar = sorted(list(char_tar))

enc_tok_num = len(char_inp)
dec_tok_num= len(char_tar)
max_encoder_seq_length = max([len(txt) for txt in inps_txt])
max_decoder_seq_length = max([len(txt) for txt in tars_txt])
val_max_encoder_seq_length = max([len(txt) for txt in val_input_texts])
val_max_decoder_seq_length = max([len(txt) for txt in val_target_texts])

In [ ]:
print("Number of samples:", len(inps_txt))
print("Number of unique input tokens:", enc_tok_num)
print("Number of unique output tokens:", dec_tok_num)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
print("Max sequence length for val inputs:", val_max_encoder_seq_length)
print("Max sequence length for val outputs:", val_max_decoder_seq_length)
print(char_inp)
print(char_tar)

Number of samples: 44204
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21
Max sequence length for val inputs: 18
Max sequence length for val outputs: 16
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[' ', 'B', 'E', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॐ']


In [ ]:
print(inps_txt[123:130])
print(tars_txt[123:130])

['jansaadharan', 'konkan', 'illingworth', 'pritish', 'bauddha', 'jaidi', 'dabav']
['BजनसाधारणE', 'BकोंकणE', 'Bइलिंगवर्थE', 'Bप्रीतिशE', 'Bबौद्धE', 'BजैदीE', 'BदबावE']


In [ ]:
tok_ind_inp= dict([(char, i) for i, char in enumerate(char_inp)])
tok_ind_tar= dict([(char, i) for i, char in enumerate(char_tar)])

In [ ]:
print(tok_ind_inp)
print(tok_ind_tar)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{' ': 0, 'B': 1, 'E': 2, 'ँ': 3, 'ं': 4, 'ः': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ए': 13, 'ऐ': 14, 'ऑ': 15, 'ओ': 16, 'औ': 17, 'क': 18, 'ख': 19, 'ग': 20, 'घ': 21, 'ङ': 22, 'च': 23, 'छ': 24, 'ज': 25, 'झ': 26, 'ञ': 27, 'ट': 28, 'ठ': 29, 'ड': 30, 'ढ': 31, 'ण': 32, 'त': 33, 'थ': 34, 'द': 35, 'ध': 36, 'न': 37, 'प': 38, 'फ': 39, 'ब': 40, 'भ': 41, 'म': 42, 'य': 43, 'र': 44, 'ल': 45, 'व': 46, 'श': 47, 'ष': 48, 'स': 49, 'ह': 50, '़': 51, 'ा': 52, 'ि': 53, 'ी': 54, 'ु': 55, 'ू': 56, 'ृ': 57, 'ॅ': 58, 'े': 59, 'ै': 60, 'ॉ': 61, 'ो': 62, 'ौ': 63, '्': 64, 'ॐ': 65}


In [ ]:
enc_inp = np.zeros(
    (len(inps_txt), max_encoder_seq_length), dtype="float32"
)
dec_inp = np.zeros(
    (len(inps_txt), max_decoder_seq_length), dtype="float32"
)
dec_tar = np.zeros(
    (len(inps_txt), max_decoder_seq_length, dec_tok_num), dtype="float32"
)

In [ ]:
for i, (txt_inp, txt_tar) in enumerate(zip(inps_txt, tars_txt)):
    
    for t, char in enumerate(txt_inp):
        enc_inp[i, t] = tok_ind_inp[char]
    enc_inp[i, t + 1 :] = tok_ind_inp[" "]

    for t, char in enumerate(txt_tar):

        dec_inp[i, t] = tok_ind_tar[char]

        if t > 0:
            dec_tar[i, t - 1, tok_ind_tar[char]] = 1.0

    dec_inp[i, t + 1: ] = tok_ind_tar[" "]
    dec_tar[i, t:, tok_ind_tar[" "]] = 1.0

In [ ]:
inp_enc_val = np.zeros(
    (len(inps_txt), val_max_encoder_seq_length), dtype="float32"
)
inp_dec_val= np.zeros(
    (len(inps_txt), val_max_decoder_seq_length), dtype="float32"
)
tar_dec_val = np.zeros(
    (len(inps_txt), val_max_decoder_seq_length, dec_tok_num), dtype="float32"
)

In [ ]:
for i, (txt_inp, txt_tar) in enumerate(zip(val_input_texts, val_target_texts)):
    
    for t, char in enumerate(txt_inp):
        inp_enc_val[i, t] = tok_ind_inp[char]
    inp_enc_val[i, t + 1 :] = tok_ind_inp[" "]

    for t, char in enumerate(txt_tar):
        inp_dec_val[i, t] = tok_ind_tar[char]

        if t > 0:
            tar_dec_val[i, t - 1, tok_ind_tar[char]] = 1.0
            
    inp_dec_val[i, t + 1: ] = tok_ind_tar[" "]
    tar_dec_val[i, t:, tok_ind_tar[" "]] = 1.0

In [ ]:
rtci= dict((i, char) for char, i in tok_ind_tar.items())

In [ ]:
x_test = inp_enc_val
y_test = val_target_texts

In [ ]:
sweep_configuration = {
    'method': 'bayes', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {

        'drop_prob': {
            'values': [0.2,0.3]
        },
        'lr': {
            'values': [1e-3, 1e-4]
        },
        'batch_size': {
            'values': [64, 128]
        },
        'input_embedding': {
            'values': [16,32, 64,128,256]
        },
        'number_of_encoders': {
            'values': [1, 2, 3]
        },
        'number_of_decoders': {
            'values': [1, 2, 3]
        },
        'hid_layer_size':{
            'values': [16,32, 64, 128,256]
        },
        'Type': {
            'values': ['rnn', 'gru', 'lstm']
        },
        'dec_search': {
            'values': ['beam_search', 'greedy']
        },
        'beam_width':{
            'values': [3,5]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_configuration, entity="shubham-argha", project="Assignment 3")

Create sweep with ID: doww1p5f
Sweep URL: https://wandb.ai/shubham-argha/Assignment%203/sweeps/doww1p5f


In [ ]:
def train():
        config_defaults = {
            'drop_prob': 0.4,
            'lr': 1e-3,
            'batch_size': 32,
            'number_of_epochs' : 15,
            'input_embedding': 32,
            'number_of_encoders': 2,
            'number_of_decoders': 2,
            'hid_layer_size': 32,
            'Type': 'rnn',
            'dec_search': 'beam_search',
            'beam_width': 5
            }

        wandb.init(config = config_defaults)
        config = wandb.config
        wandb.run.name = str(config.Type)+ '_' + config.dec_search+'_in_emb_'+str(config.input_embedding)+'_num_enc_'+str(config.number_of_encoders)+'_num_dec_'+str(config.number_of_decoders)+'_dp_'+str(config.drop_prob)+'_hidden_'+str(config.hid_layer_size)

        model_rnn = seq2seq(Type = config.Type, input_embedding = config.input_embedding, hid_layer_size=config.hid_layer_size,
                    lr= config.lr, drop_prob=config.drop_prob,prediction = config.dec_search,number_of_epochs = config.number_of_epochs,
                    batch_size = config.batch_size, beam_width = config.beam_width, number_of_encoders = config.number_of_encoders,number_of_decoders = config.number_of_decoders)

        model_rnn.fit(enc_inp,dec_inp,dec_tar,x_test, y_test)

In [ ]:
wandb.agent('fmyurg9f',train, count = 1)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: ambyaiaw with config:
wandb: 	Type: gru
wandb: 	batch_size: 64
wandb: 	beam_width: 3
wandb: 	dec_search: greedy
wandb: 	drop_prob: 0.2
wandb: 	hidden_size: 64
wandb: 	in_emb: 16
wandb: 	lr: 0.0001
wandb: 	num_dec: 3
wandb: 	num_enc: 2


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Enc_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 Enc_emb (Embedding)            (None, None, 32)     864         ['Enc_inputs[0][0]']             
                                                                                                  
 Dec_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 Enc_hidden_1 (GRU)             [(None, None, 32),   6336        ['Enc_emb[0][0]']                
                                 (None, 32)]                                                  

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


691/691 [==============================] - 53s 54ms/step - loss: 1.3631 - accuracy: 0.1632 - _timestamp: 1651399542.0000 - _runtime: 63.0000
Epoch 2/15
691/691 [==============================] - 37s 54ms/step - loss: 1.1862 - accuracy: 0.2343 - _timestamp: 1651399579.0000 - _runtime: 100.0000
Epoch 3/15
572/691 [=======================>......] - ETA: 6s - loss: 1.1084 - accuracy: 0.3030